# [課題] 高エネルギー実験で生成される荷電粒子の飛跡を、変分量子回路を使って再構成する

この課題では、高エネルギー実験では必須の技術である**「荷電粒子飛跡の再構成」**を、変分量子固有値ソルバー法を使って実現することを目指します。

```{contents} 目次
---
local: true
---
```

## はじめに<a id='introduction'></a>
**変分量子固有値ソルバー法**（*Variational Quantum Eigensolver*, VQE）を紹介した[ノートブック](vqe.ipynb)で、VQEの考え方と変分量子回路の基本的な実装の方法を学びました。ここでは、VQEの高エネルギー物理への応用を考えてみます。

高エネルギー実験では、加速器内で粒子（例えば陽子）を衝突させ、生成された多数の二次粒子を検出器で測定することで、その生成過程をつかさどる基礎物理反応を理解していきます。そのためには、検出器で測定された信号から生成粒子を同定し、そのエネルギーや運動量等を正しく再構成することがとても重要です。この実習では、生成粒子の再構成のための最初のステップとして、「荷電粒子飛跡の再構成」をVQEで実現する方法について学んでいきます。

## 高エネルギー実験<a id='hep'></a>
### LHC実験の概要<a id='hep_LHC'></a>

```{image} figs/LHC_ATLAS.png
:alt: LHC_ATLAS
:class: bg-primary mb-1
:width: 1000px
:align: center
```


LHC（大型ハドロン加速器 Large Hadron Collider）はスイスとフランスの国境に位置する欧州原子核研究機構（CERN）で運転されている円形の加速器です。地下約100 mに掘られた周長27 kmのトンネルの中に置かれ、6.5 TeVのエネルギーまで陽子を加速することができます。その加速された陽子を正面衝突させることで、世界最高エネルギーである13 TeV での陽子衝突実験を実現しています（最も左の写真）。左から2番目の写真は、地下トンネルに設置されたLHCの写真です。

LHCでは4つの実験（ATLAS, CMS, ALICE, LHCb）が進行中ですが、その中でもATLASとCMSは大型の汎用検出器を備えた実験です（右から2番目の写真が実際のATLAS検出器）。陽子衝突で発生した二次粒子を周囲に設置した高精度の検出器で観測することで、さまざまな素粒子反応の観測や新しい現象の探索などを行っています。最も右の写真は、2012年にATLASとCMSで発見されたヒッグス粒子の候補となる生成反応を示したものです。

### 荷電粒子の測定<a id='hep_detect'></a>

ATLASやCMS実験の検出器は、異なる性質を持った検出器を内側から外側に階層的に配置しています。最内層の検出器は荷電粒子の再構成や識別に使われる検出器で、実験では最も重要な検出器の一つです。この検出器はそれ自体が約10層程度の層構造を持っており、一つの荷電粒子が通過したとき、複数の検出器信号を作ります。
例えば、左下図にあるように一度の陽子衝突で無数の粒子が生成され、それらが検出器に”ヒット”と呼ばれる信号を作ります（図中の白、黄色、オレンジ等の点に相当）。このヒットの集合から「ある荷電粒子が作ったヒットの組み合わせ」を選び、その粒子の「飛跡」を再構成します。右下図のいろいろな色の曲線が飛跡に対応します。この飛跡の再構成は、ATLASやCMS実験に限らず、高エネルギー実験では最も重要な実験技術の一つと言えます。

```{image} figs/tracking.png
:alt: tracking
:class: bg-primary mb-1
:width: 600px
:align: center
```


## 飛跡の再構成<a id='tracking'></a>
この実習で目指すのは、高エネルギー粒子の衝突で発生する**荷電粒子の飛跡を再構成する**こと（**Tracking**と呼ぶ）です。ただし、現在の量子コンピュータでは大きなサイズの問題を解くことはまだ難しいため、サイズの小さい問題（つまり少数の生成粒子が生成された場合）に絞って検討を行います。


まず、必要なライブラリを最初にインポートします。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua import QuantumInstance

### TrackMLチャレンジ<a id='ML_challenge'></a>

データとしては、2018年に行われた[TrackML Particle Trackingチャレンジ](https://www.kaggle.com/c/trackml-particle-identification)で提供されたオープンデータを活用します。このチャレンジは、CERNで予定されている高輝度LHC計画（2027年に開始予定）での実験環境を想定し、Trackingを高度化するための解析技術を開発することを目指して行われました。

### [課題] ハミルトニアンの構成とVQEの実行<a id='hamiltonian_form'></a>

課題として考えるのは、Trackingを実現するためのハミルトニアンを構成し、それをVQEに実装して実行することです。

この課題ではTrackMLチャレンジのデータを用いますが、元データは扱いが難しいため、量子計算に用いやすいように前処理を行なったデータを使います。まず下図に示したように、検出器3層に渡って連続するヒットを選び出します（点線で囲まれた3層ヒットのことを、ここではセグメントと呼ぶことにします）。この時検出器中心から発生した粒子を優先的に選び出せるように、セグメントが検出器中心の方を向いているものを選びます。

```{image} figs/track_segment.png
:alt: track_segment
:class: bg-primary mb-1
:width: 350px
:align: center
```


こうして選び出したセグメントのリストを作り、そのリストから任意のペアを選んで、その“相互作用の強さ”を考えます。この相互作用は物理的な力ではなく、二つのセグメントが”同一の荷電粒子”が作る飛跡にどれぐらい近いかを表す指標だと考えてください。セグメントのペアが同一飛跡に近くなるにつれ、相互作用の強さは-1に近づくように設定されています。相互作用が+1の場合は、ヒットが二つのセグメントに共有されているため（図中で赤で示したケースに相当）、飛跡の候補としては適切でないことになります。つまり、緑で示したケースに相当するセグメントの組み合わせを選び出すことが、ここでの目標です。

この段階まで処理されたデータを、"data/QUBO_05pct_input.txt"というファイルで提供しています。ファイルの中を見ると分かりますが、個々のセグメントは"23749_38657_45525"のような名前がついており、セグメントの対をキーとする辞書型のデータとして格納されています。同じセグメントの対をキーとするデータがなぜこういう値をもっているのかは、前処理に使うモデルの詳細に依るのでここでは説明を省略します（ここでの課題には影響ありません）。

以上のデータから、VQEで用いるハミルトニアンを構成してみてください。

ヒント：この形式のデータを以下のコードを使って読み込むとします。

In [ ]:
file_r = 'data/QUBO_05pct_input.txt'
from ast import literal_eval
with open(file_r, "r") as f:
    line = f.read()
    Q = literal_eval(line)
print("Q size =",len(Q))


n_qubits = 100

nvar = 0
key_i = []
b_ij = np.zeros((n_qubits,n_qubits))
for (k1, k2), v in Q.items():
    if k1 == k2:
        b_ij[nvar][nvar] = v
        key_i.append(k1)
        nvar += 1

for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_ij[i][j] = v
                    else:
                        b_ij[j][i] = v

セグメント数を$N$とすると、この時b_ijは$N$行$N$列の正方行列になります。実はこのデータは、**QUBO**（*Quadratic Unconstrained Binary Optimization*、2次制約無し2値最適化）と呼ばれる問題として解くことができる形式で与えられています。QUBOは量子ビット$T$がバイナリー値（0か1）を持つ場合に、以下の式で与えられる目的関数$O$を最小化する問題として定義されます。

$$
O(a,b,T) = \sum_{i=1}^Nb_{ii}T_i + \sum_{i=1}^N\sum_{j=1\:(i<j)}^Nb_{ij}T_iT_j
$$

$T$は{0,1}のバイナリー値を持ちますが、シンプルな計算で{+1,-1}を持つ量子ビットに変換することができます。{+1,-1}はパウリ$Z$演算子の固有値でもあるため、パウリ$Z$演算子を使ってハミルトニアンを構成することができれば、そのまま量子回路に実装することができます。

以下のスペースに、どのような変換が可能か等を含め、VQEでTrackingを実行するために必要な量子演算を定義してください。

In [ ]:
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator

def get_qubitops(...):
    ...
    return ...

### VQEによる近似解の探索<a id='tracking_vqe'></a>

上で定義したハミルトニアンを元に、VQEを使ってエネルギーの最小固有値（の近似解）を求めていきます。ただその前に、このハミルトニアンの行列を対角化してエネルギーの最小固有値とその固有ベクトルを厳密に計算した場合の答えを出してみましょう。

In [ ]:
qubitOp = get_qubitops(...)
print("")
print("total number of qubits = ",qubitOp.num_qubits)

# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver(qubitOp)
result = ee.run()

print('Eigensolver: objective =', result.eigenvalue.real)
x = sample_most_likely(result.eigenstate)
print('Eigensolver: x =',x)

samples_eigen = {}
for i in range(nvar):
    samples_eigen[key_i[i]] = x[i]

xのリストで1になっている量子ビット（セグメント）が、最小エネルギーに対応するものとして選ばれているのが分かります。

次に、同じハミルトニアンモデルをVQEに実装して、最小エネルギーを求めてみます。オプティマイザーとしてSPSAあるいはCOBYLAを使う場合のコードは以下のようになります。

In [ ]:
# --- run optimization with VQE
seed = 10598
spsa = SPSA(maxiter=300)
cobyla = COBYLA(maxiter=500)
two = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', 'linear', reps=1)
print(two)

#backend = Aer.get_backend('statevector_simulator')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend=backend, shots=1024, seed_simulator=seed)
vqe = VQE(qubitOp, two, spsa)
#vqe = VQE(qubitOp, two, cobyla)
result = vqe.run(quantum_instance)

print('')
#print(result['optimal_parameters'])
print('VQE: objective =', result.eigenvalue.real)
x = sample_most_likely(result.eigenstate)
print('VQE x =',x)

samples_vqe = {}
for i in range(nvar):
    samples_vqe[key_i[i]] = x[i]

### おまけ<a id='omake'></a>

Trackingがうまく行っても、この答えでは面白くないですよね。正しく飛跡が見つかったかどうか目で確認するため、以下のコードを走らせてみましょう。

このコードは、QUBOを定義する時に使った検出器のヒット位置をビーム軸に垂直な平面でプロットして、どのヒットが選ばれたかを分かりやすく可視化したものです。緑の線が実際に見つかった飛跡で、青の線を含めたものが全体の飛跡の候補です。この実習では限られた数の量子ビットしか使っていないため、大部分の飛跡は見つけられていませんが、緑の線から計算に使った3点ヒットからは正しく飛跡が見つかっていることが分かると思います。

In [ ]:
from hepqpr.qallse import *
input_path = './data/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

# get the results
#all_doublets = Qallse.process_sample(samples_eigen)
all_doublets = Qallse.process_sample(samples_vqe)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)
#print("all_doublets =",all_doublets)
#print("final_tracks =",final_tracks)
#print("final_doublets =",final_doublets)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)


## 参考文献<a id='references'></a>
1. Nielsen, Michael A and Chuang, Isaac L, "Quantum Computation and Quantum Information", Cambridge University Pres, 2000.
2. Peruzzo, Alberto, et al., "A variational eigenvalue solver on a photonic quantum processor", [Nature commun. 5, 4213 (2014)](https://www.nature.com/articles/ncomms5213).

